### Faculdade de Engenharia Industrial - FEI

### Centro Universitário da Fundação Educacional Inaciana "Padre Sabóia de Medeiros" (FEI)


*FEI's Stricto Sensu Graduate Program in Electrical Engineering*

Concentration area: ARTIFICIAL INTELLIGENCE APPLIED TO AUTOMATION AND ROBOTICS

Master's thesis student Andre Luiz Florentino

## Select the processor type

In [1]:
# Select the way to load tensorflow in order to run on CPU only or CPU + GPU

import os

opc = 0
while str(opc) not in '12':
    print()
    print("1-) CPU only")
    print("2-) CPU + GPU")

    opc = input("\nSelect the processor: ")
    if opc.isdigit():
        opc = int(opc)
    else:
        opc = 0

if opc == 1:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    
    import tensorflow as tf

else:
    import tensorflow as tf
    print(tf.__version__)

    pd = tf.config.experimental.list_physical_devices()
    for i in pd:
        print(i)
    print('------------------------------------------------------------------------------------------')


    print(tf.config.list_physical_devices('GPU'))
    # [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

    print(tf.test.is_built_with_cuda)
    # <function is_built_with_cuda at 0x000001AA24AFEC10>

    print(tf.test.gpu_device_name())
    # /device:GPU:0

    #gvd = tf.config.get_visible_devices()
    for j in tf.config.get_visible_devices():
        print(j)
    # PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
    # PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

    #physical_devices = tf.config.experimental.list_physical_devices()
    #tf.config.experimental.set_memory_growth(physical_devices[2], True)


1-) CPU only
2-) CPU + GPU

Select the processor: 1


## Check for GPU

In [2]:
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

No GPU found


In [3]:
#Unload Tensorflow 
del tf

# Chapter 14: Recording real time audio

## Import modules

In [4]:
import pyaudio
import wave
import time
import threading

import IPython.display as ipd
import tkinter         as tk

from dataclasses import dataclass, asdict

In [5]:
# Globals
current_path = os.getcwd()

# For the picture names
pic_first_name = '14_Recording_real_time_audio'

In [6]:
cache_audio = os.path.join(current_path, '_cache_record_audio')
os_name = os.name

if os_name == 'posix':
    CHUNK = 4096
else:
    CHUNK = 1024
    
print(f'\nChunk size {CHUNK}\n')

# Check if the folder exists, if not, create it
if not os.path.exists(cache_audio):
    os.makedirs(cache_audio)
    
FORMAT   = pyaudio.paInt16
CHANNELS = 1
RATE     = 44100


Chunk size 1024



### Identifying the audio devices

In [7]:
# Find audio device index

devices = []
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    devices.append(p.get_device_info_by_index(i))

p.terminate()

In [8]:
devices

[{'index': 0,
  'structVersion': 2,
  'name': 'Microsoft Sound Mapper - Input',
  'hostApi': 0,
  'maxInputChannels': 2,
  'maxOutputChannels': 0,
  'defaultLowInputLatency': 0.09,
  'defaultLowOutputLatency': 0.09,
  'defaultHighInputLatency': 0.18,
  'defaultHighOutputLatency': 0.18,
  'defaultSampleRate': 44100.0},
 {'index': 1,
  'structVersion': 2,
  'name': 'Microphone (WordForum USB   )',
  'hostApi': 0,
  'maxInputChannels': 1,
  'maxOutputChannels': 0,
  'defaultLowInputLatency': 0.09,
  'defaultLowOutputLatency': 0.09,
  'defaultHighInputLatency': 0.18,
  'defaultHighOutputLatency': 0.18,
  'defaultSampleRate': 44100.0},
 {'index': 2,
  'structVersion': 2,
  'name': 'Microphone Array (Intel® Smart ',
  'hostApi': 0,
  'maxInputChannels': 4,
  'maxOutputChannels': 0,
  'defaultLowInputLatency': 0.09,
  'defaultLowOutputLatency': 0.09,
  'defaultHighInputLatency': 0.18,
  'defaultHighOutputLatency': 0.18,
  'defaultSampleRate': 44100.0},
 {'index': 3,
  'structVersion': 2,
  'n

In [9]:
devices[6]

{'index': 6,
 'structVersion': 2,
 'name': 'Stereo Mix (Realtek HD Audio Stereo input)',
 'hostApi': 1,
 'maxInputChannels': 2,
 'maxOutputChannels': 0,
 'defaultLowInputLatency': 0.01,
 'defaultLowOutputLatency': 0.01,
 'defaultHighInputLatency': 0.04,
 'defaultHighOutputLatency': 0.04,
 'defaultSampleRate': 48000.0}

In [10]:
devices[1]

{'index': 1,
 'structVersion': 2,
 'name': 'Microphone (WordForum USB   )',
 'hostApi': 0,
 'maxInputChannels': 1,
 'maxOutputChannels': 0,
 'defaultLowInputLatency': 0.09,
 'defaultLowOutputLatency': 0.09,
 'defaultHighInputLatency': 0.18,
 'defaultHighOutputLatency': 0.18,
 'defaultSampleRate': 44100.0}

In [11]:
devices[1]['name']

'Microphone (WordForum USB   )'

In [12]:
@dataclass

class StreamParams:

    input_device_index: int  = 1
    format:             int  = FORMAT
    channels:           int  = CHANNELS
    rate:               int  = RATE
    frames_per_buffer:  int  = CHUNK
    input:              bool = True
    output:             bool = False
    

    def to_dict(self) -> dict:
        return asdict(self)


class Recorder:
    """Recorder uses the blocking I/O facility from pyaudio to record sound from mic.

    Attributes:
        - stream_params: StreamParams object with values for pyaudio Stream object
    """
    def __init__(self, stream_params: StreamParams) -> None:
        self.stream_params = stream_params
        self._pyaudio      = None
        self._stream       = None
        self._wav_file     = None

    def record(self, duration: int, save_path: str) -> None:
        """Record sound from mic for a given amount of seconds.

        :param duration: Number of seconds we want to record for
        :param save_path: Where to store recording
        """
        print("Start recording...")
        self._create_recording_resources(save_path)
        self._write_wav_file_reading_from_stream(duration)
        self._close_recording_resources()
        print("Stop recording")

    def _create_recording_resources(self, save_path: str) -> None:
        self._pyaudio = pyaudio.PyAudio()
        self._stream = self._pyaudio.open(**self.stream_params.to_dict())
        self._create_wav_file(save_path)

    def _create_wav_file(self, save_path: str):
        self._wav_file = wave.open(save_path, "wb")
        self._wav_file.setnchannels(self.stream_params.channels)
        self._wav_file.setsampwidth(self._pyaudio.get_sample_size(self.stream_params.format))
        self._wav_file.setframerate(self.stream_params.rate)

    def _write_wav_file_reading_from_stream(self, duration: int) -> None:
        for _ in range(int(self.stream_params.rate * duration / self.stream_params.frames_per_buffer)):
            audio_data = self._stream.read(self.stream_params.frames_per_buffer)
            self._wav_file.writeframes(audio_data)

    def _close_recording_resources(self) -> None:
        self._wav_file.close()
        self._stream.close()
        self._pyaudio.terminate()

In [15]:
if __name__ == "__main__":
    stream_params = StreamParams()
    recorder = Recorder(stream_params)
    recorder.record(5, os.path.join(cache_audio, "audio.wav"))

Start recording...
Stop recording


In [16]:
ipd.Audio(os.path.join(cache_audio,'audio.wav'))

### Recording $n$ seconds

In [ ]:
def recording_audio():

    p = pyaudio.PyAudio()

    stream = p.open(format             = FORMAT, 
                    channels           = CHANNELS, 
                    rate               = RATE, 
                    input              = True, 
                    input_device_index = devices[1]['index'],
                    frames_per_buffer  = CHUNK)

    frames  = []
    seconds = 5

    print (f"start recording for {seconds}...")

    for i in range(0, int(RATE / CHUNK * seconds)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("recording stopped")

    stream.stop_stream()
    stream.close()
    p.terminate()
    
    return frames

In [ ]:
frames = recording_audio()

### Writing the audio file

In [ ]:
wf = wave.open(os.path.join(cache_audio, "output.wav"), "wb")
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

In [ ]:
ipd.Audio(os.path.join(cache_audio, 'output.wav'))

### Playing the audio file from the array "frames"

In [ ]:
def playing_audio(frames):
    
    play=pyaudio.PyAudio()
    
    stream_play=play.open(format    = FORMAT,
                          channels  = CHANNELS,
                          rate      = RATE,
                          output    = True)
    for data in frames: 
        stream_play.write(data)
        
    stream_play.stop_stream()
    stream_play.close()
    play.terminate()

In [ ]:
playing_audio(frames)